In [5]:
!wget http://

In [1]:
import numpy as np
import os
import sys
import time
import pandas as pd 
from tqdm._tqdm_notebook import tqdm_notebook
import pickle
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers import LSTM
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras import optimizers
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
import glob
from scipy import stats

<ipython-input-1-f8f31a5d41c3>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [6]:
hasgpu = tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

print(hasgpu)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger("tensorflow").setLevel(logging.ERROR)
os.environ['TZ'] = 'Asia/Kolkata'  # to set timezone; needed when running on cloud
#time.tzset()

params = {
    "batch_size": 32,  # 20<16<10, 25 was a bust
    "epochs": 100,
    "lr": 0.00010000,
    "time_steps": 20
}

iter_changes = "dropout_layers_0.4_0.4"

PATH_TO_DRIVE_ML_DATA = "C:/Users/carls/work/medium-stock-lstm/"

INPUT_PATH = PATH_TO_DRIVE_ML_DATA+"data"
OUTPUT_PATH = PATH_TO_DRIVE_ML_DATA+"outputs/lstm_best_2020-fx/"+iter_changes
TIME_STEPS = params["time_steps"]
BATCH_SIZE = params["batch_size"]
stime = time.time()

# check if directory already exists
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
    print("Directory created", OUTPUT_PATH)
#else:
#    raise Exception("Directory already exists. Don't override.")


def print_time(text, stime):
    seconds = (time.time()-stime)
    print(text, seconds//60,"minutes : ",np.round(seconds%60),"seconds")


def trim_dataset(mat,batch_size):
    """
    trims dataset to a size that's divisible by BATCH_SIZE
    """
    no_of_rows_drop = mat.shape[0]%batch_size
    if no_of_rows_drop > 0:
        return mat[:-no_of_rows_drop]
    else:
        return mat
    
def build_lstm_timeseries(mat, proc):
    """
    Converts ndarray into timeseries format and supervised data format. Takes first TIME_STEPS
    number of rows as input and sets the TIME_STEPS+1th data as corresponding output and so on.
    :param mat: ndarray which holds the dataset
    :param y_col_index: index of column which acts as output
    :return: returns two ndarrays-- input and output in format suitable to feed
    to LSTM.
    """
    # total number of time-series samples would be len(mat) - TIME_STEPS
    dim_0 = mat.shape[0] - TIME_STEPS
    dim_1 = mat.shape[1]
    forward_test = 1
    x = np.zeros((dim_0-forward_test, TIME_STEPS, dim_1))
    #y = np.zeros((dim_0-forward_test , forward_test))
    y = np.zeros(dim_0-forward_test)
    print("dim_0",dim_0)
    ct = 0
    for i in tqdm_notebook(range(dim_0-forward_test)):
        x[ct] = mat[i:TIME_STEPS+i]
        # set the value after n timesteps, for open and close
        #for r in range(forward_test):
        #    y[ct][r] = mat[i+TIME_STEPS+r, 0]
        y[ct] = mat[i+TIME_STEPS, 0]
        ct = ct + 1
        
    return x, y


True
Directory created C:/Users/carls/work/medium-stock-lstm/outputs/lstm_best_2020-fx/dropout_layers_0.4_0.4


In [7]:
def relative_strength(prices, n=14):

    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1. + rs)

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # cause the diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n - 1) + upval)/n
        down = (down*(n - 1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1. + rs)

    return rsi

def avg_slopes(prices, n=5):
    deltas = np.diff(prices)
    
    slopes = np.zeros_like(prices)
    
    for i in range(n,len(prices)):
        delta1 = deltas[i - 1]
        delta2 = deltas[i - 2]
        delta3 = deltas[i - 3]
        delta4 = deltas[i - 4]
        
        slope = (delta1 + delta2 + delta3 + delta4) / 4.0
        
        slopes[i] = slope
    
    return slopes

def candle_legs(opens, closes, highs, lows ):
    ctop = np.zeros_like(opens)
    cbot = np.zeros_like(opens)
    
    _opens = opens.values
    _closes = closes.values
    _highs = highs.values
    _lows = lows.values
    
    for i in range(len(opens)):
        bodyhigh = max(_opens[i],_closes[i])
        bodymin = min(_opens[i],_closes[i])
        
        if (_highs[i] > bodyhigh):
            ctop[i] = _highs[i] - bodyhigh
        else:
            ctop[i] = 0.0
        
        if (_lows[i] < bodymin):
            cbot[i] = _lows[i] - bodymin
        else:
            cbot[i] = 0.0
    
    return ctop, cbot

In [10]:
## get csv files
import datetime as dt
from sklearn import linear_model

MMScalers = []
SeriesData_x = []
SeriesData_y = []

print('start')

extension = 'csv'
all_filenames = [i for i in glob.glob(INPUT_PATH + '/*.{}'.format(extension))]
for fn in all_filenames:
    print(fn)
    stime = time.time()
    df_ge = pd.read_csv(fn, engine='python')
    
    print(df_ge.head(20))
    
    no_of_std = 2
    df_ge['return'] = df_ge['close'].pct_change(1)
    df_ge['log_return'] = np.log( df_ge['return'] + 1)
    
    df_ge['avg30_mean'] = df_ge['close'].rolling(window=26).mean()
    df_ge['avg30_std'] = df_ge['close'].rolling(window=26).std()
    df_ge['rsi'] = relative_strength(df_ge['close'])
    
    df_ge['avg30_mean_bb_high'] = df_ge['avg30_mean'] + no_of_std * df_ge['avg30_std']
    df_ge['avg30_mean_bb_low'] = df_ge['avg30_mean'] - no_of_std * df_ge['avg30_std']
    
    df_ge['avg12_mean'] = df_ge['close'].rolling(window=12).mean()
    df_ge['avg12_std'] = df_ge['close'].rolling(window=12).std()
    df_ge['avg12_mean_bb_high'] = df_ge['avg12_mean'] + 1 * df_ge['avg12_std']
    df_ge['avg12_mean_bb_low'] = df_ge['avg12_mean'] - 1 * df_ge['avg12_std']
    
    df_ge['avg12_mean_bb_high2'] = df_ge['avg12_mean'] + 2 * df_ge['avg12_std']
    df_ge['avg12_mean_bb_low2'] = df_ge['avg12_mean'] - 2 * df_ge['avg12_std']
    
    df_ge['avg12_mean_bb_high3'] = df_ge['avg12_mean'] + 3 * df_ge['avg12_std']
    df_ge['avg12_mean_bb_low3'] = df_ge['avg12_mean'] - 3 * df_ge['avg12_std']
    
    df_ge['avg5_mean'] = df_ge['close'].rolling(window=5).mean()
    
    #df_ge['up_rally'] = rally_up_count( df_ge['close'] )
    #df_ge['down_rally'] = rally_down_count( df_ge['close'] )
    
    #df_ge['up10_rally'] = rally_up_num( df_ge['close'], 20 )
    #df_ge['down10_rally'] = rally_down_num( df_ge['close'], 20 )
    
    ## now get a slope
    
    df_ge = df_ge.iloc[30:] # rem first 30 with avges
    
    # now set some slopes
    df_ge['avg30_mean_slopes'] = avg_slopes(df_ge['avg30_mean'])
    df_ge['avg12_mean_slopes'] = avg_slopes(df_ge['avg12_mean'])
    df_ge = df_ge.iloc[8:] # rem first 30 with avges
    
    df_ge['macd'] = df_ge['avg12_mean'] - df_ge['avg30_mean']
    df_ge['macd2'] = df_ge['avg5_mean'] - df_ge['avg12_mean']
    
    # candle, up or down
    #df_ge['candle_body'] = df_ge['close'] - df_ge['open']
    #ctop, cbot = candle_legs(df_ge['open'], df_ge['close'], df_ge['high'], df_ge['low'])
    
    #df_ge['candle_top'] = ctop
    #df_ge['candle_bot'] = cbot
    
    df_ge['avg30_diff'] = df_ge['close'] - df_ge['avg30_mean']
    #df_ge['avg30_bb_high_diff'] = df_ge['close'] - df_ge['avg30_mean_bb_high']
    #df_ge['avg30_bb_low_diff'] = df_ge['close'] - df_ge['avg30_mean_bb_low']
    
    df_ge['avg12_diff'] = df_ge['close'] - df_ge['avg12_mean']
    df_ge['avg12_bb_high_diff'] = df_ge['close'] - df_ge['avg12_mean_bb_high']
    df_ge['avg12_bb_low_diff'] = df_ge['close'] - df_ge['avg12_mean_bb_low']
    
    # avgs are positive or negative.. get average change over 
    
    #print(df_ge.head(20))
    train_cols = ["log_return", "avg12_diff", "avg12_bb_high_diff", "avg12_bb_low_diff"]
    
    #  MinMaxScaler(feature_range=(-3,3))
    #for key in train_cols:
    #    min_max_scaler = MinMaxScaler()
    #    scaled = min_max_scaler.fit_transform(df_ge[[key]])
    #    df_ge[key] = scaled
    #    MMScalers.append(min_max_scaler)
    
    #train_cols = ["close"]
    
    x = df_ge.loc[:,train_cols].values
    print(df_ge.loc[:,train_cols].head(20))
    
    proccols = ["change" , "rate", "open", "close"]
    xx = df_ge.loc[:,proccols].values
    #print(xx.head(20))
     
    #x_t, y_t = build_lstm_timeseries(x_train, xx)
    x_t, y_t = build_lstm_timeseries(x, xx)
    
    SeriesData_x.append(x_t)
    SeriesData_y.append(y_t)
    
print('end')

start
C:/Users/carls/work/medium-stock-lstm/data\midat_usdjpy.csv
                   date  change     rate      low     high     open    close  \
0   2016-10-02 21:15:00     2.0  101.320  101.303  101.361  101.315  101.320   
1   2016-10-02 21:30:00   -27.0  101.293  101.235  101.346  101.320  101.293   
2   2016-10-02 21:45:00    11.0  101.304  101.287  101.334  101.294  101.304   
3   2016-10-02 22:00:00   100.0  101.404  101.296  101.427  101.309  101.404   
4   2016-10-02 22:15:00   -28.0  101.376  101.356  101.404  101.402  101.376   
5   2016-10-02 22:30:00    30.0  101.406  101.350  101.419  101.391  101.406   
6   2016-10-02 22:45:00    40.0  101.446  101.408  101.471  101.411  101.446   
7   2016-10-02 23:00:00   -61.0  101.385  101.351  101.464  101.464  101.385   
8   2016-10-02 23:15:00    70.0  101.455  101.378  101.468  101.386  101.455   
9   2016-10-02 23:30:00   -30.0  101.425  101.404  101.470  101.454  101.425   
10  2016-10-02 23:45:00   103.0  101.528  101.418  101

  0%|          | 0/67602 [00:00<?, ?it/s]

end


In [9]:
# Visualize the prediction
#from matplotlib import pyplot as plt
#plt.figure()
#plt.rcParams["figure.figsize"] = (20,12)


#plt.plot(df_ge['avg12_mean'][0:50], linewidth=2.0)

#plt.plot(df_ge['avg12_mean_bb_high'][0:50], linewidth=1.0)
#plt.plot(df_ge['avg12_mean_bb_low'][0:50], linewidth=1.0)

#plt.plot(df_ge['avg12_mean_bb_high2'][0:50], linewidth=1.0)
#plt.plot(df_ge['avg12_mean_bb_low2'][0:50], linewidth=1.0)

#plt.plot(df_ge['avg12_mean_bb_high3'][0:50], linewidth=1.0)
#plt.plot(df_ge['avg12_mean_bb_low3'][0:50], linewidth=1.0)
#plt.plot(df_ge['close'][0:50], linewidth=4.0)
#plt.title('chart')
#plt.ylabel('Price')
#plt.xlabel('Days')
#plt.legend(['mean','o','-o','2o','-2o','3o','-3o','close'], loc='upper left')
#plt.show()

In [10]:
from sklearn.utils import shuffle
# now put all of our series data together and shuffle

TRAIN_RATIO = 0.75
TRAIN_SIZE = int(len(x_t) * TRAIN_RATIO)

x_t = SeriesData_x.pop(0)
y_t = SeriesData_y.pop(0)

print(f'total records:{len(x_t)}')

# shuffle, then split into test and train
#x_t, y_t = shuffle(x_t, y_t, random_state=0)

final_test_x = x_t[TRAIN_SIZE:]
final_train_x = x_t[:TRAIN_SIZE]

final_test_y = y_t[TRAIN_SIZE:]
final_train_y = y_t[:TRAIN_SIZE]

print(final_train_x[0:14])
print(final_train_y[0:14])

print(f'final_train_x:{final_train_x.shape}')

##series_test_x = SeriesData_x.pop(-1)
##series_train_x = SeriesData_x.pop(0)
##for arr in SeriesData_x:
##    series_train_x = np.concatenate((series_train_x , arr))

##series_test_y = SeriesData_y.pop(-1)
##series_train_y = SeriesData_y.pop(0)
##for arr in SeriesData_y:
##    series_train_y = np.concatenate((series_train_y , arr))

##final_train_x = series_train_x[:]
##final_test_x = series_test_x[:]

##final_train_y = series_train_y[:]
##final_test_y = series_test_y[:]

x_t = trim_dataset(final_train_x, BATCH_SIZE)
y_t = trim_dataset(final_train_y, BATCH_SIZE)

x_val, x_test_t = np.split(trim_dataset(final_test_x, BATCH_SIZE),2)
y_val, y_test_t = np.split(trim_dataset(final_test_y, BATCH_SIZE),2)

x_val, y_val = shuffle(x_val, y_val, random_state=0)

#x_val = trim_dataset(final_test_x, BATCH_SIZE),2)
#y_val = trim_dataset(final_test_y, BATCH_SIZE),2)

print(x_t.shape)
print(x_val.shape)

total records:67602
[[[5.77700751e-04 6.24357472e-01 7.87170207e-01 4.50523617e-01]
  [1.79087233e-03 6.32510101e-01 7.93734392e-01 4.59356005e-01]
  [7.04794916e-03 6.57613503e-01 8.14716966e-01 4.85608190e-01]
  ...
  [6.29693819e-03 6.50202023e-01 8.17824208e-01 4.66453631e-01]
  [5.66146736e-03 6.48098118e-01 8.16774103e-01 4.63384942e-01]
  [6.23916811e-03 6.51803859e-01 8.20808652e-01 4.66111401e-01]]

 [[1.79087233e-03 6.32510101e-01 7.93734392e-01 4.59356005e-01]
  [7.04794916e-03 6.57613503e-01 8.14716966e-01 4.85608190e-01]
  [9.24321202e-03 6.67224520e-01 8.22098703e-01 4.96457846e-01]
  ...
  [5.66146736e-03 6.48098118e-01 8.16774103e-01 4.63384942e-01]
  [6.23916811e-03 6.51803859e-01 8.20808652e-01 4.66111401e-01]
  [3.46620451e-03 6.39539054e-01 8.10529343e-01 4.53319441e-01]]

 [[7.04794916e-03 6.57613503e-01 8.14716966e-01 4.85608190e-01]
  [9.24321202e-03 6.67224520e-01 8.22098703e-01 4.96457846e-01]
  [5.89254766e-03 6.51325699e-01 8.08934341e-01 4.79678706e-01]
  ..

In [11]:
def create_lstm_model():
    lstm_model = Sequential()
    
    #lstm_model.add(LSTM(units = 50,return_sequences=True,input_shape = (TIME_STEPS,  x_t.shape[2])) )
    lstm_model.add(LSTM(units = 50, return_sequences = True, 
                        batch_input_shape=(BATCH_SIZE, TIME_STEPS, x_t.shape[2]), 
                        stateful=True ))
    lstm_model.add(Dropout(0.2))
    #lstm_model.add(BatchNormalization())
    lstm_model.add(LSTM(units = 50, return_sequences = True))
    lstm_model.add(Dropout(0.2))
    
    lstm_model.add(LSTM(units = 50))
    lstm_model.add(Dropout(0.2))
    #lstm_model.add(BatchNormalization())
    
    #lstm_model.add(Dense(32, activation='relu'))
    #lstm_model.add(Dropout(0.2))
    
    lstm_model.add(Dense(1,activation='sigmoid'))
    # optimizer = optimizers.RMSprop(lr=params["lr"])
    # optimizer = optimizers.SGD(lr=0.000001, decay=1e-6, momentum=0.9, nesterov=True)
    optimizer = optimizers.Adam(lr=params["lr"], decay=1e-6)
    #lstm_model.compile(loss='mean_squared_error', optimizer=optimizer)
    lstm_model.compile(loss='mean_squared_error', optimizer=optimizer)
    return lstm_model

In [12]:
model = None
#try:
    #model = pickle.load(open("chart_model", 'rb'))
    #print("Loaded saved model...")
#except FileNotFoundError:
#    print("Model not found")


#x_temp, y_temp = build_timeseries(x_test, 3)
#x_val, x_test_t = np.split(trim_dataset(x_temp, BATCH_SIZE),2)
#y_val, y_test_t = np.split(trim_dataset(y_temp, BATCH_SIZE),2)

print("Test size", x_test_t.shape, y_test_t.shape, x_val.shape, y_val.shape)
    
is_update_model = True
if model is None or is_update_model:
    from keras import backend as K
    print("Building model...")
    model = create_lstm_model()
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,
                       patience=40, min_delta=0.0001)
    
    mcp = ModelCheckpoint(OUTPUT_PATH + "/best_chart_model.h5", monitor='val_loss', verbose=1,
                          save_best_only=True, save_weights_only=False, mode='min', period=1)

    # Not used here. But leaving it here as a reminder for future
    r_lr_plat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=30, 
                                  verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
    
    csv_logger = CSVLogger((OUTPUT_PATH + '/training_log_1.log'))
    
    history = model.fit(x_t, y_t, epochs=params["epochs"], verbose=2, batch_size=BATCH_SIZE,
                        shuffle=False, validation_data=(trim_dataset(x_val, BATCH_SIZE),
                        trim_dataset(y_val, BATCH_SIZE)), callbacks=[es, mcp, csv_logger])
    
    print("saving model...")
    #pickle.dump(model, open("chart_model", "wb"))

Test size (8448, 20, 4) (8448,) (8448, 20, 4) (8448,)
Building model...
Epoch 1/100
1584/1584 - 16s - loss: 0.0086 - val_loss: 0.0138

Epoch 00001: val_loss improved from inf to 0.01379, saving model to C:/Users/carls/work/jp-stock-pred-rnn/outputs/lstm_best_2020-fx/dropout_layers_0.4_0.4\best_chart_model.h5
Epoch 2/100
1584/1584 - 12s - loss: 0.0041 - val_loss: 0.0071

Epoch 00002: val_loss improved from 0.01379 to 0.00708, saving model to C:/Users/carls/work/jp-stock-pred-rnn/outputs/lstm_best_2020-fx/dropout_layers_0.4_0.4\best_chart_model.h5
Epoch 3/100
1584/1584 - 12s - loss: 0.0021 - val_loss: 0.0054

Epoch 00003: val_loss improved from 0.00708 to 0.00536, saving model to C:/Users/carls/work/jp-stock-pred-rnn/outputs/lstm_best_2020-fx/dropout_layers_0.4_0.4\best_chart_model.h5
Epoch 4/100
1584/1584 - 12s - loss: 0.0016 - val_loss: 0.0051

Epoch 00004: val_loss improved from 0.00536 to 0.00508, saving model to C:/Users/carls/work/jp-stock-pred-rnn/outputs/lstm_best_2020-fx/dropout

1584/1584 - 12s - loss: 3.2897e-04 - val_loss: 5.6843e-04

Epoch 00037: val_loss improved from 0.00062 to 0.00057, saving model to C:/Users/carls/work/jp-stock-pred-rnn/outputs/lstm_best_2020-fx/dropout_layers_0.4_0.4\best_chart_model.h5
Epoch 38/100
1584/1584 - 12s - loss: 3.2885e-04 - val_loss: 5.7695e-04

Epoch 00038: val_loss did not improve from 0.00057
Epoch 39/100
1584/1584 - 12s - loss: 3.2415e-04 - val_loss: 5.7300e-04

Epoch 00039: val_loss did not improve from 0.00057
Epoch 40/100
1584/1584 - 12s - loss: 3.2501e-04 - val_loss: 5.2257e-04

Epoch 00040: val_loss improved from 0.00057 to 0.00052, saving model to C:/Users/carls/work/jp-stock-pred-rnn/outputs/lstm_best_2020-fx/dropout_layers_0.4_0.4\best_chart_model.h5
Epoch 41/100
1584/1584 - 12s - loss: 3.2127e-04 - val_loss: 4.7005e-04

Epoch 00041: val_loss improved from 0.00052 to 0.00047, saving model to C:/Users/carls/work/jp-stock-pred-rnn/outputs/lstm_best_2020-fx/dropout_layers_0.4_0.4\best_chart_model.h5
Epoch 42/100
1

1584/1584 - 12s - loss: 2.4459e-04 - val_loss: 1.7986e-04

Epoch 00083: val_loss did not improve from 0.00011
Epoch 84/100
1584/1584 - 12s - loss: 2.4979e-04 - val_loss: 1.7521e-04

Epoch 00084: val_loss did not improve from 0.00011
Epoch 85/100
1584/1584 - 12s - loss: 2.4152e-04 - val_loss: 1.4539e-04

Epoch 00085: val_loss did not improve from 0.00011
Epoch 86/100
1584/1584 - 13s - loss: 2.4188e-04 - val_loss: 1.3467e-04

Epoch 00086: val_loss did not improve from 0.00011
Epoch 87/100
1584/1584 - 13s - loss: 2.4199e-04 - val_loss: 1.3485e-04

Epoch 00087: val_loss did not improve from 0.00011
Epoch 88/100
1584/1584 - 12s - loss: 2.3413e-04 - val_loss: 1.3468e-04

Epoch 00088: val_loss did not improve from 0.00011
Epoch 89/100
1584/1584 - 12s - loss: 2.3266e-04 - val_loss: 1.7054e-04

Epoch 00089: val_loss did not improve from 0.00011
Epoch 90/100
1584/1584 - 12s - loss: 2.2881e-04 - val_loss: 1.7984e-04

Epoch 00090: val_loss did not improve from 0.00011
Epoch 91/100
1584/1584 - 12s 

In [13]:

y_pred = model.predict(trim_dataset(x_test_t, BATCH_SIZE), batch_size=BATCH_SIZE)

min_max_scaler = MMScalers[0]
y_test_t = trim_dataset(y_test_t, BATCH_SIZE)
error = mean_squared_error(y_test_t, y_pred)
print("Error is", error, y_pred.shape, y_test_t.shape)

print(min_max_scaler.data_range_[0] , min_max_scaler.data_min_[0])

# convert the predicted value to range of real data
y_pred_org = (y_pred * min_max_scaler.data_range_[0]) + min_max_scaler.data_min_[0]
# min_max_scaler.inverse_transform(y_pred)
y_test_t_org = (y_test_t * min_max_scaler.data_range_[0]) + min_max_scaler.data_min_[0]

print(y_pred_org[5:50])
print(y_test_t_org[5:50])

Error is 8.862622519826532e-05 (8448, 1) (8448,)
17.309999999999988 101.263
[[111.549614]
 [111.69906 ]
 [111.866035]
 [111.699   ]
 [111.758545]
 [111.81626 ]
 [111.6971  ]
 [111.78245 ]
 [111.77596 ]
 [111.9056  ]
 [111.6757  ]
 [111.70974 ]
 [111.67782 ]
 [111.59294 ]
 [111.790924]
 [111.68111 ]
 [111.845085]
 [111.82865 ]
 [111.575096]
 [111.820076]
 [111.69986 ]
 [111.660286]
 [111.777794]
 [111.6646  ]
 [111.73753 ]
 [111.80773 ]
 [111.857315]
 [111.70536 ]
 [111.71894 ]
 [111.749695]
 [111.71889 ]
 [111.715126]
 [111.7148  ]
 [111.732   ]
 [111.73721 ]
 [111.69808 ]
 [111.702866]
 [111.7173  ]
 [111.70262 ]
 [111.713615]
 [111.70129 ]
 [111.71799 ]
 [111.68314 ]
 [111.68123 ]
 [111.673256]]
[111.529 111.529 111.529 111.537 111.537 111.537 111.546 111.546 111.546
 111.52  111.52  111.52  111.497 111.497 111.497 111.517 111.517 111.517
 111.525 111.525 111.525 111.522 111.522 111.522 111.527 111.527 111.527
 111.566 111.566 111.566 111.566 111.566 111.566 111.534 111.534 111.534
 